# Dual Arm Motion Planning 🦾🦾

## 1. Setup 🏗️️

### 1.1 Recreating the ICRA 2024 Cloth Competition Scene 👕

In [1]:
import numpy as np
from airo_drake import DualArmScene, add_floor, add_manipulator, add_meshcat, add_wall, finish_build
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.tree import ModelInstanceIndex
from pydrake.planning import RobotDiagramBuilder, SceneGraphCollisionChecker
import airo_models


def add_cloth_competition_dual_ur5e_scene(
    robot_diagram_builder: RobotDiagramBuilder,
    arm_left_transform: RigidTransform,
    arm_right_transform: RigidTransform,
    add_cylinder: bool = False,
) -> tuple[tuple[ModelInstanceIndex, ModelInstanceIndex], tuple[ModelInstanceIndex, ModelInstanceIndex]]:
    plant = robot_diagram_builder.plant()
    parser = robot_diagram_builder.parser()
    parser.SetAutoRenaming(True)

    add_floor(robot_diagram_builder, y_size=2.4)

    # Arms places side by side on the table
    arm_y = arm_left_transform.translation()[1]

    # Add three safety walls
    wall_thickness = 0.2
    wall_left_position = np.array([0, arm_y + 0.7 + wall_thickness / 2, 0])
    wall_right_position = np.array([0, -arm_y - 0.7 - wall_thickness / 2, 0])
    wall_back_position = np.array([0.9 + wall_thickness / 2, 0, 0])
    add_wall(robot_diagram_builder, "XZ", 2.0, 2.0, wall_thickness, wall_left_position, "wall_left")
    add_wall(robot_diagram_builder, "XZ", 2.0, 2.0, wall_thickness, wall_right_position, "wall_right")
    add_wall(robot_diagram_builder, "YZ", 2.0, 2.7, wall_thickness, wall_back_position, "wall_back")

    if add_cylinder:
        cylinder_urdf_path = airo_models.cylinder_urdf_path(length=0.4, radius=0.1, name="cloth")
        cylinder_index = parser.AddModels(cylinder_urdf_path)[0]
        cylinder_transform = RigidTransform(p=[0, 0, 0.6])
        world_frame = plant.world_frame()
        cylinder_frame = plant.GetFrameByName("base_link", cylinder_index)
        plant.WeldFrames(world_frame, cylinder_frame, cylinder_transform)

    # The robot arms
    arm_left_index, gripper_left_index = add_manipulator(
        robot_diagram_builder, "ur5e", "robotiq_2f_85", arm_left_transform, static_gripper=True
    )
    arm_right_index, gripper_right_index = add_manipulator(
        robot_diagram_builder, "ur5e", "robotiq_2f_85", arm_right_transform, static_gripper=True
    )

    return (arm_left_index, arm_right_index), (gripper_left_index, gripper_right_index)

In [2]:
robot_diagram_builder = RobotDiagramBuilder()

meshcat = add_meshcat(robot_diagram_builder)
meshcat.SetCameraPose([-1.5, 0, 1.0], [0, 0, 0])

arm_y = 0.45
X_W_L = RigidTransform(rpy=RollPitchYaw([0, 0, np.pi / 2]), p=[0, arm_y, 0])
X_W_R = RigidTransform(rpy=RollPitchYaw([0, 0, np.pi / 2]), p=[0, -arm_y, 0])


(arm_left_index, arm_right_index), (gripper_left_index, gripper_right_index) = add_cloth_competition_dual_ur5e_scene(
    robot_diagram_builder, X_W_L, X_W_R
)
robot_diagram, context = finish_build(robot_diagram_builder, meshcat)

scene = DualArmScene(robot_diagram, arm_left_index, arm_right_index, gripper_left_index, gripper_right_index, meshcat)

INFO:drake:Meshcat listening for connections at http://localhost:7000
LCM self test failed!!
Check your routing tables and firewall settings


### 1.2 Dual arm `SceneGraphCollisionChecker` 💥💥

The only diffrence when making a `SceneGraphCollisionChecker` for dual arm is you need to provide it additional `ModelInstanceIndex`s.  

In [3]:
collision_checker = SceneGraphCollisionChecker(
    model=scene.robot_diagram,
    robot_model_instances=[
        scene.arm_left_index,
        scene.arm_right_index,
        scene.gripper_left_index,
        scene.gripper_right_index,
    ],
    edge_step_size=0.125,  # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding=0.005,
    self_collision_padding=0.005,
)

INFO:drake:Allocating contexts to support implicit context parallelism 32


In [4]:
scene.robot_diagram.plant().num_positions()

12

In [5]:
collision_checker.CheckConfigCollisionFree(np.zeros(12))

False

In [6]:
home_joints_left = np.deg2rad([180, -120, 60, -30, -90, -90])
home_joints_right = np.deg2rad([-180, -60, -60, -150, 90, 90])

home_joints = np.concatenate([home_joints_left, home_joints_right])

collision_checker.CheckConfigCollisionFree(home_joints)

True

In [23]:
plant = scene.robot_diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)

plant.SetPositions(plant_context, home_joints)
scene.robot_diagram.ForcedPublish(context)  # updates the meshcat visualization

### 1.3 Inverse kinematics for both arms 🧮🧮
We will wrap the individual IK functions to express TCP poses in a common world frame.

In [8]:
from functools import partial
from airo_typing import HomogeneousMatrixType, JointConfigurationType
from airo_drake import X_URBASE_ROSBASE
from ur_analytic_ik import ur5e

tcp_transform = np.identity(4)
tcp_transform[2, 3] = 0.175


def inverse_kinematics_in_world_fn(
    tcp_pose: HomogeneousMatrixType, X_W_CB: HomogeneousMatrixType
) -> list[JointConfigurationType]:
    X_W_TCP = tcp_pose
    X_CB_W = np.linalg.inv(X_W_CB)
    solutions_1x6 = ur5e.inverse_kinematics_with_tcp(X_CB_W @ X_W_TCP, tcp_transform)
    solutions = [solution.squeeze() for solution in solutions_1x6]
    return solutions


# This conversion is needed because the URDFs use the ROS base frame convention:
X_CB_B = X_URBASE_ROSBASE
X_W_LCB = X_W_L.GetAsMatrix4() @ np.linalg.inv(X_CB_B.GetAsMatrix4())
X_W_RCB = X_W_R.GetAsMatrix4() @ np.linalg.inv(X_CB_B.GetAsMatrix4())

inverse_kinematics_left_fn = partial(inverse_kinematics_in_world_fn, X_W_CB=X_W_LCB)
inverse_kinematics_right_fn = partial(inverse_kinematics_in_world_fn, X_W_CB=X_W_RCB)

### 1.4 `DualArmOmplPlanner` 🧭

In [9]:
from airo_planner import DualArmOmplPlanner

# These joint bounds are specifically for UR5e mounted on a table.
# joint_bounds_lower = np.deg2rad([-360, -195, -160, -360, -360, -360])
# joint_bounds_upper = np.deg2rad([360, 15, 160, 360, 360, 360])
joint_bounds = None


planner = DualArmOmplPlanner(
    is_state_valid_fn=collision_checker.CheckConfigCollisionFree,
    inverse_kinematics_left_fn=inverse_kinematics_left_fn,
    inverse_kinematics_right_fn=inverse_kinematics_right_fn,
    joint_bounds_left=joint_bounds,
    joint_bounds_right=joint_bounds,
)

We use the terms "left" and "right" for the two arms, note that you can choose yourself which arm is which, however you must take care to be consistent which arms configuration is first and which is second in the `is_state_valid_fn`. We recommend for simplificity to choose as left arm the arm that also appear on the left side of image from the robot's primary camera, as this is how an egocentric robot would name its arms.

The API for dual arm motion planning is very similar to the single arm API, expect that you can supply two joint configurations or two tcp poses instead of one. 

## 2. Scenarios 🖼️

In this section we demonstrate the `airo-planner` API by going over the main Cloth Competition use cases.

TODO:
- scene without cloth obstacle go home
- scene grasp highest (plan to single TCP pose)
- scene with cloth obstacle grasp single arm and filter -> pregrasp no collision, grasp can collide
- scene go to stretch poses (plan to dual TCP poses)

### 2.1 Joints to joints (🦾,🦾) -> (🦾,🦾)

In [10]:
tangled_joints_left = np.deg2rad([0, -90, -90, -90, 90, 0])
tangled_joints_right = np.deg2rad([-136, -116, -110, -133, 40, 0])
tangled_joints = np.concatenate([tangled_joints_left, tangled_joints_right])

plant.SetPositions(plant_context, tangled_joints)
scene.robot_diagram.ForcedPublish(context)  # updates the meshcat visualization

In [11]:
path = planner.plan_to_joint_configuration(
    tangled_joints_left, tangled_joints_right, home_joints_left, home_joints_right
)
print(path.shape)

(5, 12)Debug:   RRTConnect: Planner range detected to be 8.706237

Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 5 states (3 start + 2 goal)
Info:    Solution found in 0.121101 seconds
Info:    SimpleSetup: Path simplification took 0.201483 seconds and changed from 3 to 3 states


In [12]:
from airo_drake import time_parametrize_toppra, animate_dual_joint_trajectory

trajectory = time_parametrize_toppra(scene.robot_diagram.plant(), path)

animate_dual_joint_trajectory(
    scene.meshcat, scene.robot_diagram, scene.arm_left_index, scene.arm_right_index, trajectory
)

### 2.2 Joints to single arm joints (🦾,🦾) -> (🦾,💤)

Note that you are allow to set one of the goals to `None` to indicate that that arm should not move.

In [13]:
path = planner.plan_to_joint_configuration(tangled_joints_left, tangled_joints_right, home_joints_left, None)

Debug:   RRTConnect: Planner range detected to be 6.156239
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 4 states (2 start + 2 goal)
Info:    Solution found in 0.014816 seconds
Info:    SimpleSetup: Path simplification took 0.134128 seconds and changed from 3 to 3 states


2025-04-16 12:27:00.955 | SUCCESS  | airo_planner.ompl.single_arm_ompl_planner:plan_to_joint_configuration:129 - Successfully found path (with 65 waypoints).


In [14]:
trajectory = time_parametrize_toppra(scene.robot_diagram.plant(), path)
animate_dual_joint_trajectory(
    scene.meshcat, scene.robot_diagram, scene.arm_left_index, scene.arm_right_index, trajectory
)

### 2.3 Joints to single TCP pose  (🦾,🦾) -> (💤,📐)

In [15]:
from airo_drake import animate_joint_configurations, visualize_frame

grasp_pose_hard = np.identity(4)
grasp_pose_hard[2, 3] = 0.4

visualize_frame(scene.meshcat, "grasp_pose", grasp_pose_hard)

grasp_joints = inverse_kinematics_right_fn(grasp_pose_hard)

animate_joint_configurations(scene.meshcat, scene.robot_diagram, scene.arm_right_index, grasp_joints, context=context)

In [16]:
path = planner.plan_to_tcp_pose(tangled_joints_left, home_joints_right, None, grasp_pose_hard)

2025-04-16 12:27:01.072 | INFO     | airo_planner.ompl.single_arm_ompl_planner:_calculate_ik_solutions:156 - IK returned 8 solutions.
2025-04-16 12:27:01.073 | INFO     | airo_planner.multiple_goal_planner:_check_ik_solutions_bounds:102 - Found 8/8 solutions within joint bounds.
2025-04-16 12:27:01.076 | INFO     | airo_planner.multiple_goal_planner:_check_ik_solutions_validity:58 - Found 3/8 valid solutions.
2025-04-16 12:27:01.076 | INFO     | airo_planner.multiple_goal_planner:plan_to_ik_solutions:156 - Running OMPL towards 3 IK solutions.


Debug:   RRTConnect: Planner range detected to be 6.156239
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 4 states (2 start + 2 goal)


2025-04-16 12:27:01.171 | SUCCESS  | airo_planner.ompl.single_arm_ompl_planner:plan_to_joint_configuration:129 - Successfully found path (with 2 waypoints).


Info:    Solution found in 0.028481 seconds
Info:    SimpleSetup: Path simplification took 0.065318 seconds and changed from 3 to 2 states
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 7 states (5 start + 2 goal)
Info:    Solution found in 0.060038 seconds


2025-04-16 12:27:01.368 | SUCCESS  | airo_planner.ompl.single_arm_ompl_planner:plan_to_joint_configuration:129 - Successfully found path (with 2 waypoints).


Info:    SimpleSetup: Path simplification took 0.135879 seconds and changed from 4 to 2 states
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 7 states (5 start + 2 goal)

2025-04-16 12:27:01.646 | SUCCESS  | airo_planner.ompl.single_arm_ompl_planner:plan_to_joint_configuration:129 - Successfully found path (with 65 waypoints).
2025-04-16 12:27:01.647 | SUCCESS  | airo_planner.multiple_goal_planner:_plan_to_goal_configurations:134 - Found 3 paths towards IK solutions, (planning time: 0.57 s).
2025-04-16 12:27:01.647 | SUCCESS  | airo_planner.multiple_goal_planner:_plan_to_goal_configurations:143 - Chose path with 2 waypoints.



Info:    Solution found in 0.048472 seconds
Info:    SimpleSetup: Path simplification took 0.186969 seconds and changed from 4 to 3 states


In [17]:
trajectory = time_parametrize_toppra(scene.robot_diagram.plant(), path)
animate_dual_joint_trajectory(
    scene.meshcat, scene.robot_diagram, scene.arm_left_index, scene.arm_right_index, trajectory
)

### 2.3 Joints to dual TCP poses  (🦾,🦾) -> (📐,📐)

In [19]:
from airo_typing import RotationMatrixType


def hang_in_the_air_tcp_orientation(left: bool) -> RotationMatrixType:
    gripper_forward_direction = np.array([0, -1, 0]) if left else np.array([0, 1, 0])
    Z = gripper_forward_direction / np.linalg.norm(gripper_forward_direction)
    X = np.array([0, 0, 1]) if left else np.array([0, 0, -1])
    Y = np.cross(Z, X)
    return np.column_stack([X, Y, Z])


def hang_in_the_air_tcp_pose(left: bool) -> HomogeneousMatrixType:
    position = np.array([0, 0, 0.9])  # 1 m is too close to a singularity
    gripper_orientation = hang_in_the_air_tcp_orientation(left)

    gripper_pose = np.identity(4)
    gripper_pose[0:3, 0:3] = gripper_orientation
    gripper_pose[0:3, 3] = position
    return gripper_pose


hang_pose_left = hang_in_the_air_tcp_pose(left=True)
hang_pose_right = hang_in_the_air_tcp_pose(left=False)

stretch_pose_left = hang_pose_left.copy()
stretch_pose_right = hang_pose_right.copy()

stretch_pose_left[:3, 3] += np.array([-0.4, 0.02, -.3])
stretch_pose_right[:3, 3] += np.array([-0.4, -0.02, -.3])

visualize_frame(scene.meshcat, "stretch_pose_left", stretch_pose_left, opacity=0.5)
visualize_frame(scene.meshcat, "stretch_pose_right", stretch_pose_right, opacity=0.5)

In [20]:
path = planner.plan_to_tcp_pose(home_joints_left, home_joints_right, stretch_pose_left, stretch_pose_right)

2025-04-16 12:28:16.108 | INFO     | airo_planner.ompl.dual_arm_ompl_planner:_calculate_ik_solutions:387 - IK returned 64 pairs of IK solutions (8 x 8).
2025-04-16 12:28:16.108 | INFO     | airo_planner.multiple_goal_planner:_check_ik_solutions_bounds:102 - Found 64/64 solutions within joint bounds.
2025-04-16 12:28:16.119 | INFO     | airo_planner.multiple_goal_planner:_check_ik_solutions_validity:58 - Found 49/64 valid solutions.
2025-04-16 12:28:16.119 | INFO     | airo_planner.multiple_goal_planner:plan_to_ik_solutions:156 - Running OMPL towards 49 IK solutions.


Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 5 states (2 start + 3 goal)
Info:    Solution found in 0.194809 seconds
Info:    SimpleSetup: Path simplification took 0.114252 seconds and changed from 3 to 3 states
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 10 states (5 start + 5 goal)
Info:    Solution found in 0.301121 seconds
Info:    SimpleSetup: Path simplification took 0.332091 seconds and changed from 7 to 8 states
Info:    RRTConnect: Starting planning with 1 states already in datastructure

2025-04-16 12:28:19.089 | INFO     | airo_planner.multiple_goal_planner:_plan_to_goal_configurations:120 - No path found to goal configuration: 3.



Info:    RRTConnect: Created 12 states (5 start + 7 goal)
Info:    Solution found in 0.659980 seconds
Info:    SimpleSetup: Path simplification took 0.180667 seconds and changed from 5 to 2 states
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 12 states (5 start + 7 goal)
Info:    ProblemDefinition: Adding approximate solution from planner RRTConnect
Info:    Solution found in 1.011189 seconds
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 5 states (2 start + 3 goal)
Info:    Solution found in 0.097995 seconds
Info:    SimpleSetup: Path simplification took 0.116159 seconds and changed from 4 to 3 states
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 5 states (3 start + 2 goal)
Info:    Solution found in 0.081113 seconds
Info:    SimpleSetup: Path simplification took 0.082293 seconds and changed from 3 to 3 s

2025-04-16 12:28:21.809 | INFO     | airo_planner.multiple_goal_planner:_plan_to_goal_configurations:120 - No path found to goal configuration: 10.


...

2025-04-16 12:28:34.207 | INFO     | airo_planner.multiple_goal_planner:_plan_to_goal_configurations:120 - No path found to goal configuration: 34.
2025-04-16 12:28:36.041 | INFO     | airo_planner.multiple_goal_planner:_plan_to_goal_configurations:120 - No path found to goal configuration: 38.


Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 9 states (5 start + 4 goal)
Info:    ProblemDefinition: Adding approximate solution from planner RRTConnect
Info:    Solution found in 1.010476 seconds
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 4 states (2 start + 2 goal)
Info:    Solution found in 0.035915 seconds
Info:    SimpleSetup: Path simplification took 0.115250 seconds and changed from 3 to 2 states
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 4 states (2 start + 2 goal)
Info:    Solution found in 0.037997 seconds
Info:    SimpleSetup: Path simplification took 0.072470 seconds and changed from 3 to 2 states
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 5 states (2 start + 3 goal)
Info:    Solution found in 0.400384 seconds
Info:    Simpl

2025-04-16 12:28:38.219 | INFO     | airo_planner.multiple_goal_planner:_plan_to_goal_configurations:120 - No path found to goal configuration: 45.


Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 4 states (2 start + 2 goal)
Info:    Solution found in 0.049547 seconds
Info:    SimpleSetup: Path simplification took 0.076182 seconds and changed from 3 to 2 states
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 4 states (2 start + 2 goal)
Info:    Solution found in 0.043191 seconds
Info:    SimpleSetup: Path simplification took 0.088725 seconds and changed from 3 to 2 states
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 4 states (2 start + 2 goal)
Info:    Solution found in 0.040040 seconds
Info:    SimpleSetup: Path simplification took 0.065186 seconds and changed from 3 to 2 states
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 14 states (7 start + 7 goal)
Info:    ProblemDefinition: Adding approxi

2025-04-16 12:28:39.041 | SUCCESS  | airo_planner.multiple_goal_planner:_plan_to_goal_configurations:134 - Found 36 paths towards IK solutions, (planning time: 22.92 s).
2025-04-16 12:28:39.042 | SUCCESS  | airo_planner.multiple_goal_planner:_plan_to_goal_configurations:143 - Chose path with 2 waypoints.


In [21]:
path.shape

(2, 12)

In [22]:
trajectory = time_parametrize_toppra(scene.robot_diagram.plant(), path)
animate_dual_joint_trajectory(
    scene.meshcat, scene.robot_diagram, scene.arm_left_index, scene.arm_right_index, trajectory
)

In [21]:
scene.meshcat.Delete("stretch_pose_left")
scene.meshcat.Delete("stretch_pose_right")

### 2.4 Joints to single TCP pose with ranking (🦾,🦾) -> (💤,📐)

In [ ]:
from airo_planner import rank_by_distance_to_desirable_configurations
from airo_planner import stack_joints

# Note how we set desirable left joints to it's start configuration as the left arm is not involved in the task
desirable_configurations = [stack_joints(tangled_joints_left, home_joints_right)]

rank_fn = partial(rank_by_distance_to_desirable_configurations, desirable_configurations=desirable_configurations)


planner.rank_goal_configurations_fn = rank_fn
path = planner.plan_to_tcp_pose(tangled_joints_left, tangled_joints_right, None, hang_pose_right)
planner.rank_goal_configurations_fn = None  # Remove the rank function for when other cells are run

In [23]:
trajectory = time_parametrize_toppra(scene.robot_diagram.plant(), path)
animate_dual_joint_trajectory(
    scene.meshcat, scene.robot_diagram, scene.arm_left_index, scene.arm_right_index, trajectory
)

## 3. Advanced filtering and ranking for grasping ⚙️

In this example we show the full capability of airo-planner for a complex cloth competition use cases, grasping a hanging cloth with one arm that is held in the air with another arm.

We want to approach the grasp pose linearly from pregrasp pose that lies a few cm behind it (several distances can be tried). The path to the pregrasp pose should not collide with the convex hull of the hanging cloth. The path from the pregrasp pose to the grasp pose should not collide with the environment, but is allowed to contact the cloth. 

Additionally, of all solution paths, we prefer paths where the linear grasp motion can be executed "comfortably", so we rank the pregrasp joints by the TOPP-RA calculated duration of the grasp trajectory.

To summarize:
* We will choose a pregrasp distance and calculate the resulting pregrasp pose and grasp TCP path.
* We calculate the environment-collision-free joint paths that can execute this TCP path.
* We calculate their duration and rank the pregrasp joints according to grasp path duration
* We try to plan a cloth-collision-free pregrasp path to the pregrasp joints   
* If anything fails, try a different pregrasp distance
* If all pregrasp distances fail, consider the grasp pos unreachable.

### 3.1 Filtering collision

In [ ]:
robot_diagram_builder = RobotDiagramBuilder()
meshcat = add_meshcat(robot_diagram_builder)
meshcat.SetCameraPose([-1.5, 0, 1.0], [0, 0, 0])

(arm_left_index, arm_right_index), (gripper_left_index, gripper_right_index) = add_cloth_competition_dual_ur5e_scene(
    robot_diagram_builder, X_W_L, X_W_R, add_cylinder=True
)
robot_diagram, context = finish_build(robot_diagram_builder, meshcat)

scene_with_cloth = DualArmScene(
    robot_diagram,
    arm_left_index,
    arm_right_index,
    gripper_left_index,
    gripper_right_index,
    meshcat,
)

In [ ]:
hang_pose_left = hang_in_the_air_tcp_pose(left=True)

hang_joints_left = inverse_kinematics_left_fn(hang_pose_left)

for joints in hang_joints_left:
    print(joints)


start_joints_left = rank_by_distance_to_desirable_configurations(hang_joints_left, [home_joints_left])[0]

print("Chosen:")
print(start_joints_left)

In [26]:
start_joints_right = home_joints_right
start_joints = np.concatenate([start_joints_left, start_joints_right])

plant = robot_diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)
plant.SetPositions(plant_context, start_joints)
robot_diagram.ForcedPublish(context)  # updates the meshcat visualization

Visualization of plannig without the cloth obstacle

In [ ]:
path = planner.plan_to_tcp_pose(start_joints_left, start_joints_right, None, grasp_pose_hard)
trajectory = time_parametrize_toppra(scene.robot_diagram.plant(), path)
animate_dual_joint_trajectory(
    scene_with_cloth.meshcat,
    scene_with_cloth.robot_diagram,
    scene_with_cloth.arm_left_index,
    scene_with_cloth.arm_right_index,
    trajectory,
)

In [ ]:
collision_checker_with_cloth = SceneGraphCollisionChecker(
    model=scene_with_cloth.robot_diagram,
    robot_model_instances=[
        scene_with_cloth.arm_left_index,
        scene_with_cloth.arm_right_index,
        scene_with_cloth.gripper_left_index,
        scene_with_cloth.gripper_right_index,
    ],
    edge_step_size=0.125,  # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding=0.005,
    self_collision_padding=0.005,
)

In [ ]:
collision_checker_with_cloth.CheckConfigCollisionFree(trajectory.value(trajectory.end_time()))

In [30]:
is_state_valid_fn_pregrasp = collision_checker_with_cloth.CheckConfigCollisionFree
is_state_valid_fn_grasp = collision_checker.CheckConfigCollisionFree

In [31]:
planner_pregrasp = DualArmOmplPlanner(
    is_state_valid_fn=is_state_valid_fn_pregrasp,
    inverse_kinematics_left_fn=inverse_kinematics_left_fn,
    inverse_kinematics_right_fn=inverse_kinematics_right_fn,
    joint_bounds_left=joint_bounds,
    joint_bounds_right=joint_bounds,
)

In [32]:
from airo_typing import InverseKinematicsFunctionType, JointConfigurationCheckerType, JointPathType
from loguru import logger
from pydrake.trajectories import PiecewisePose, Trajectory
from pydrake.math import RigidTransform
from airo_drake import discretize_drake_pose_trajectory
from airo_drake import calculate_valid_joint_paths
from functools import partial
from airo_planner import filter_with_distance_to_configurations, PlannerError
from pydrake.multibody.plant import MultibodyPlant
from airo_drake import concatenate_drake_trajectories


# TODO consider making a class for this that stores debug information
def plan_pregrasp_and_grasp_trajetory(
    planner_pregrasp: DualArmOmplPlanner,
    grasp_pose: HomogeneousMatrixType,
    start_configuration_left: JointConfigurationType,
    start_configuration_right: JointConfigurationType,
    inverse_kinematics_left_fn: InverseKinematicsFunctionType,  # same comment as for is_state_valid_fn_grasp
    inverse_kinematics_right_fn: InverseKinematicsFunctionType,
    is_state_valid_fn_grasp: JointConfigurationCheckerType,  # could make this optional and use planner's by default
    plant_toppra: MultibodyPlant,
    with_left: bool = True,
) -> tuple[Trajectory]:

    # We add 1.0 so at least one pregrasp distance fails:
    pregrasp_distances_to_try = [0.05, 0.1, 0.15]  # , 0.2, 0.25]

    # is_state_valid_fn_grasp currently still takes a 12-DoF configuration
    def is_single_arm_state_valid_fn_grasp(joint_configuration: JointConfigurationType) -> bool:
        if with_left:
            return is_state_valid_fn_grasp(stack_joints(joint_configuration, start_configuration_right))
        else:
            return is_state_valid_fn_grasp(stack_joints(start_configuration_left, joint_configuration))

    def hardcoded_cost_fn(
        joint_configuration: JointConfigurationType,
        known_joint_configurations: list[JointConfigurationType],
        costs: list[float],
    ) -> float:
        distances = [
            np.linalg.norm(joint_configuration - known_configuration)
            for known_configuration in known_joint_configurations
        ]
        if np.min(distances) > 0.001:
            logger.warning(f"Joint configuration is not close to any known configurations. {joint_configuration} ")
        return costs[np.argmin(distances)]

    def rank_with_cost_fn(
        joint_configurations: list[JointConfigurationType], cost_fn: JointConfigurationCheckerType
    ) -> list[JointConfigurationType]:
        return sorted(joint_configurations, key=cost_fn)

    for distance in pregrasp_distances_to_try:
        logger.info(f"Planning to pregrasp pose at distance {distance}.")
        # 1. Compute pregrasp pose
        pregrasp_pose = grasp_pose.copy()
        pregrasp_pose[0:3, 3] -= distance * pregrasp_pose[0:3, 2]

        pregrasp_pose_left = pregrasp_pose if with_left else None
        pregrasp_pose_right = pregrasp_pose if not with_left else None

        # 2. Compute grasp TCP path
        rigid_transforms = [RigidTransform(pose) for pose in [pregrasp_pose, grasp_pose]]
        times = np.linspace(0, 1, len(rigid_transforms))
        pose_trajectory = PiecewisePose.MakeLinear(times=times, poses=rigid_transforms)
        grasp_tcp_path = discretize_drake_pose_trajectory(pose_trajectory).poses

        # 3 Compute valid grasp joint paths
        inverse_kinematics_fn = inverse_kinematics_left_fn if with_left else inverse_kinematics_right_fn

        grasp_path_single_arm = calculate_valid_joint_paths(
            grasp_tcp_path, inverse_kinematics_fn, is_single_arm_state_valid_fn_grasp
        )

        if len(grasp_path_single_arm) == 0:
            logger.info(f"No valid grasp joint paths found for distance {distance}, continuing to next distance.")
            continue

        if with_left:
            grasp_paths = [stack_joints(path, start_configuration_right) for path in grasp_path_single_arm]
        else:
            grasp_paths = [stack_joints(start_configuration_left, path) for path in grasp_path_single_arm]

        # Create filter function
        grasp_path_starts = [path[0] for path in grasp_paths]
        filter_fn = partial(filter_with_distance_to_configurations, joint_configurations_close=grasp_path_starts)

        # Create rank function
        grasp_trajectories = []
        times = []
        for path in grasp_paths:
            trajectory = time_parametrize_toppra(plant_toppra, path)
            times.append(trajectory.end_time())
            grasp_trajectories.append(trajectory)

        cost_fn = partial(hardcoded_cost_fn, known_joint_configurations=grasp_path_starts, costs=times)
        rank_fn = partial(rank_with_cost_fn, cost_fn=cost_fn)

        # Plan
        planner_pregrasp.filter_goal_configurations_fn = filter_fn
        planner_pregrasp.rank_goal_configurations_fn = rank_fn

        try:
            pregrasp_path = planner_pregrasp.plan_to_tcp_pose(
                start_configuration_left, start_configuration_right, pregrasp_pose_left, pregrasp_pose_right
            )
        except PlannerError as e:
            logger.info(
                f"Failed to plan to pregrasp pose at distance {distance}, continuing to next distance. Exception was:\n {e}."
            )
            continue

        pregrasp_trajectory = time_parametrize_toppra(plant_toppra, pregrasp_path)

        # # Find the grasp trajectory of which the start is closest to the pregrasp path end (=pregrasp end joints)
        # We will likely want an airo-planner helper function for this
        pregrasp_end_joints = pregrasp_path[-1]
        distances = [np.linalg.norm(start - pregrasp_end_joints) for start in grasp_path_starts]
        index_of_closest_start = np.argmin(distances)

        assert np.isclose(distances[index_of_closest_start], 0, atol=0.01)  # sanity check

        grasp_trajectory = grasp_trajectories[index_of_closest_start]

        # final set: concatenate pregrasp and grasp trajectories
        pregrasp_and_grasp_trajectory = concatenate_drake_trajectories([pregrasp_trajectory, grasp_trajectory])

        return pregrasp_and_grasp_trajectory

    raise RuntimeError("Grasp planner exhausted all pregrasp poses to try")

In [33]:
from airo_drake import visualize_frame

grasp_location = np.array([0.0, 0.0, 0.4])

gripper_forward_direction = np.array([1, 0, 0])

Z = gripper_forward_direction / np.linalg.norm(gripper_forward_direction)
Y = np.array([0, 0, -1])  # 0, 0, 1 is also an option
X = np.cross(Y, Z)

grasp_orientation = np.column_stack([X, Y, Z])
grasp_pose_easy = np.identity(4)
grasp_pose_easy[0:3, 0:3] = grasp_orientation
grasp_pose_easy[0:3, 3] = grasp_location

visualize_frame(scene_with_cloth.meshcat, "grasp_pose_easy", grasp_pose_easy, length=0.25)

In [ ]:
pregrasp_and_grasp_trajectory_easy = plan_pregrasp_and_grasp_trajetory(
    planner_pregrasp,
    grasp_pose_easy,
    start_joints_left,
    start_joints_right,
    inverse_kinematics_left_fn,
    inverse_kinematics_right_fn,
    is_state_valid_fn_grasp,
    plant,
    with_left=False,
)

animate_dual_joint_trajectory(
    scene_with_cloth.meshcat,
    scene_with_cloth.robot_diagram,
    scene_with_cloth.arm_left_index,
    scene_with_cloth.arm_right_index,
    pregrasp_and_grasp_trajectory_easy,
)

In [ ]:
scene_with_cloth.meshcat.Delete("grasp_pose_easy")

In [ ]:
grasp_location = np.array([-0.08, 0.0, 0.33])

gripper_forward_direction = np.array([1, -1, 0])

Z = gripper_forward_direction / np.linalg.norm(gripper_forward_direction)
Y = np.array([0, 0, -1])  # 0, 0, 1 is also an option
X = np.cross(Y, Z)

grasp_orientation = np.column_stack([X, Y, Z])
grasp_pose_hard = np.identity(4)
grasp_pose_hard[0:3, 0:3] = grasp_orientation
grasp_pose_hard[0:3, 3] = grasp_location

visualize_frame(scene_with_cloth.meshcat, "grasp_pose_hard", grasp_pose_hard, length=0.25)

In [ ]:
pregrasp_and_grasp_trajectory_hard = plan_pregrasp_and_grasp_trajetory(
    planner_pregrasp,
    grasp_pose_hard,
    start_joints_left,
    start_joints_right,
    inverse_kinematics_left_fn,
    inverse_kinematics_right_fn,
    is_state_valid_fn_grasp,
    plant,
    with_left=False,
)

animate_dual_joint_trajectory(
    scene_with_cloth.meshcat,
    scene_with_cloth.robot_diagram,
    scene_with_cloth.arm_left_index,
    scene_with_cloth.arm_right_index,
    pregrasp_and_grasp_trajectory_hard,
)

In [ ]:
scene_with_cloth.meshcat.Delete("grasp_pose_hard")